In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


def fetch_latest_8k_filing_with_details(cik):
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    }

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    if 'filings' in data and 'recent' in data['filings']:
        recent_filings = data['filings']['recent']
        forms = recent_filings.get('form', [])
        report_dates = recent_filings.get('reportDate', [])
        filing_dates = recent_filings.get('filingDate', [])
        accession_numbers = recent_filings.get('accessionNumber', [])
        primary_documents = recent_filings.get('primaryDocument', [])
        descriptions = recent_filings.get('primaryDocDescription', [])
        acceptance_date_times = recent_filings.get('acceptanceDateTime', [])
        acts = recent_filings.get('act', [])
        file_numbers = recent_filings.get('fileNumber', [])
        film_numbers = recent_filings.get('filmNumber', [])
        items = recent_filings.get('items', [])
        sizes = recent_filings.get('size', [])
        is_xbrl = recent_filings.get('isXBRL', [])
        is_inline_xbrl = recent_filings.get('isInlineXBRL', [])

        latest_filing = None
        latest_report_date = None

        for i, form in enumerate(forms):
            if form == "8-K":
                if latest_report_date is None or report_dates[i] > latest_report_date:
                    latest_report_date = report_dates[i]
                    latest_filing = {
                        "form": form,
                        "filing_date": filing_dates[i],
                        "report_date": report_dates[i],
                        "accession_number": accession_numbers[i],
                        "primary_document": primary_documents[i],
                        "description": descriptions[i],
                        "acceptance_date_time": acceptance_date_times[i],
                        "act": acts[i],
                        "file_number": file_numbers[i],
                        "film_number": film_numbers[i],
                        "items": items[i],
                        "size": sizes[i],
                        "is_xbrl": is_xbrl[i],
                        "is_inline_xbrl": is_inline_xbrl[i]
                    }

        return latest_filing
    else:
        print("Recent filings not found in the response.")
        return None


def scrape_filing_details(filing_url):
    response = requests.get(filing_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    company_name = None
    reverse_split_date = None

    # Search for the company name
    for text in soup.stripped_strings:
        if 'inc.' in text.lower() and 'name' in text.lower():
            company_name = text.split('Inc.')[0].strip() + ' Inc.'
            break

    # Search for the reverse split date
    for text in soup.stripped_strings:
        if 'date of report' in text.lower() or 'date of earliest event reported' in text.lower():
            reverse_split_date = text.split(':')[-1].strip()
            break

    return company_name, reverse_split_date


# Nuburu, Inc. CIK
cik = "0001760689"

# Fetch and print the latest report date for 8-K filings
latest_filing = fetch_latest_8k_filing_with_details(cik)

if latest_filing:
    filing_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{latest_filing['accession_number'].replace('-', '')}/{latest_filing['primary_document']}"
    company_name, reverse_split_date = scrape_filing_details(filing_url)

    print(f"Form: {latest_filing['form']}")
    print(f"Accession Number: {latest_filing['accession_number']}")
    print(f"Filing Date: {latest_filing['filing_date']}")
    print(f"Report Date: {latest_filing['report_date']}")
    print(f"Acceptance DateTime: {latest_filing['acceptance_date_time']}")
    print(f"Act: {latest_filing['act']}")
    print(f"File Number: {latest_filing['file_number']}")
    print(f"Film Number: {latest_filing['film_number']}")
    print(f"Items: {latest_filing['items']}")
    print(f"Size: {latest_filing['size']}")
    print(f"isXBRL: {latest_filing['is_xbrl']}")
    print(f"isInlineXBRL: {latest_filing['is_inline_xbrl']}")
    print(f"Primary Document: {latest_filing['primary_document']}")
    print(f"Primary Doc Description: {latest_filing['description']}")
    print(f"Link: {filing_url}")
    print(f"Company Name: {company_name}")
    print(f"Reverse Split Date: {reverse_split_date}")
else:
    print("No 8-K filings found.")


/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Form: 8-K
Accession Number: 0001760689-24-000021
Filing Date: 2024-06-04
Report Date: 2024-06-03
Acceptance DateTime: 2024-06-03T17:31:46.000Z
Act: 34
File Number: 001-38826
Film Number: 241015128
Items: 1.01,2.03,3.02,9.01
Size: 2202750
isXBRL: 1
isInlineXBRL: 1
Primary Document: mvst-20240603.htm
Primary Doc Description: 8-K
Link: https://www.sec.gov/Archives/edgar/data/0001760689/000176068924000021/mvst-20240603.htm
Company Name: None
Reverse Split Date: None


In [ ]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


def fetch_8k_filings_with_details(cik):
    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    }

    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()

    if 'filings' in data and 'recent' in data['filings']:
        recent_filings = data['filings']['recent']
        forms = recent_filings.get('form', [])
        report_dates = recent_filings.get('reportDate', [])
        filing_dates = recent_filings.get('filingDate', [])
        accession_numbers = recent_filings.get('accessionNumber', [])
        primary_documents = recent_filings.get('primaryDocument', [])
        descriptions = recent_filings.get('primaryDocDescription', [])

        filtered_filings = []

        for i, form in enumerate(forms):
            if form == "8-K":
                filtered_filings.append({
                    "form": form,
                    "filing_date": filing_dates[i],
                    "report_date": report_dates[i],
                    "accession_number": accession_numbers[i],
                    "primary_document": primary_documents[i],
                    "description": descriptions[i],
                })

        return filtered_filings
    else:
        print("Recent filings not found in the response.")
        return []


def scrape_filing_details(filing_url):
    response = requests.get(filing_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    contains_keywords = False
    company_name = None
    reverse_split_date = None

    # Search for the company name
    for text in soup.stripped_strings:
        if 'inc.' in text.lower() and 'name' in text.lower():
            company_name = text.split('Inc.')[0].strip() + ' Inc.'
            break

    # Search for keywords "round" and "reverse split"
    for text in soup.stripped_strings:
        if 'round' in text.lower() or 'reverse split' in text.lower():
            contains_keywords = True
        if 'date of report' in text.lower() or 'date of earliest event reported' in text.lower():
            reverse_split_date = text.split(':')[-1].strip()

    return company_name, reverse_split_date, contains_keywords


# FORD CIK
cik = "0000037996"

# Fetch and print the 8-K filings containing "round" and "reverse split"
filings = fetch_8k_filings_with_details(cik)

for filing in filings:
    filing_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{filing['accession_number'].replace('-', '')}/{filing['primary_document']}"
    company_name, reverse_split_date, contains_keywords = scrape_filing_details(filing_url)

    if contains_keywords:
        print(f"Form: {filing['form']}")
        print(f"Accession Number: {filing['accession_number']}")
        print(f"Filing Date: {filing['filing_date']}")
        print(f"Report Date: {filing['report_date']}")
        print(f"Primary Document: {filing['primary_document']}")
        print(f"Primary Doc Description: {filing['description']}")
        print(f"Link: {filing_url}")
        print(f"Company Name: {company_name}")
        print(f"Reverse Split Date: {reverse_split_date}")
        print("\n" + "-"*50 + "\n")
